In [1]:
import json
import csv
import pickle
import nltk
import string
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import text
from sklearn.manifold import TSNE

from collections import defaultdict
from functools import partial

import gensim
from gensim import corpora
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
punc = set(string.punctuation)
for sw in punc:
    stops.add(sw)

dir_ = r"C:\Users\Broccoli\Documents\Bayes Hack"
#hp2014 = pickle.load(open(dir_ + r"\housing_pipeline_2014_comments.pkl", "rb"))
hi2014 = pickle.load(open(dir_ + r"\housing_inventory_2014_comments.pkl", "rb"))
hi2013 = pickle.load(open(dir_ + r"\housing_inventory_2013_comments.pkl", "rb"))
hi2012 = pickle.load(open(dir_ + r"\housing_inventory_2012_comments.pkl", "rb"))
hi2011 = pickle.load(open(dir_ + r"\housing_inventory_2012_comments.pkl", "rb"))

comments = {}
for d in [hi2014,hi2013,hi2012,hi2011]:
    for k,v in d.items():
        comments[k] = v

In [2]:
def iter_comments(comments=comments):
    for prj in comments:
        for article in comments[prj]:
            for c in article['comments']:
                yield ' '.join([''.join(ch for ch in w if ch not in punc).lower().replace('\n','') for w in c.split(' ')])

com2prj = []
for prj in comments:
    for article in comments[prj]:
        [com2prj.append(prj) for c in range(0,len(article['comments']))]

In [3]:
def sentiment_pred(sentence):
    from nltk.corpus import opinion_lexicon
    from nltk.tokenize import treebank
#    with open(dir_+r'\negative.txt', 'r') as f:
#        negative = set(f.read().split('\n'))
    negative = opinion_lexicon.negative()
    tokenizer = treebank.TreebankWordTokenizer()
    pos_words = 0
    neg_words = 0
    tokenized_sent = [word.lower() for word in tokenizer.tokenize(sentence) if word not in stops]
    x = list(range(len(tokenized_sent))) # x axis for the plot
    y = []
    for word in tokenized_sent:
        if word in opinion_lexicon.positive():
            pos_words += 1
            y.append(1) # positive
        elif any(n in word for n in negative):
            neg_words += 1
            y.append(-1) # negative
        else:
            y.append(0) # neutral
    if pos_words > neg_words:
        return 1
    elif pos_words < neg_words:
        return -1
    elif pos_words == neg_words:
        return 0

In [8]:
topics = [
            ["safe","secur","hurt","crime","protect","thie","graffiti","drug","attack","arrest","danger","murder","theft","violen","rate","fear"],
            ["congestion","parking","space","car","more","street","garage","spot"],
            ["bike","transit","muni","cycl","commut","mta","bus","bart","walk","subway","transportation","bart","station"],
            ["homeless","grim","problem","encamp","bum","ghetto","alcohol"],
            ["school","kid","famil","public","activit","stud","class","private"],
            ["green","emission","gas","environment","natur","noise","traffic","mph","pollution"],
            ["prop","13","landlord","ellis","evict","rent","subsid","tenant","pay","market","afford","bmr"],
            ["ceqa","development","approv","review","appeal","commission","permit"],
            ["height","tower","high","stor","apartment","tall","dens","bonus"],
            ["register","vote","tax","elect","ballot","bond","politic","campaign"],
            ["elite","compan","gentrif","rich","tech","wealth"]
        ]

def topic_sentiment_scores(projects, comments=comments):
    for prj in comments:
            topic_scores = []
            sentiment = 0
            if comments[prj]:
                projects[prj] = {'comments': list(), 'number_of_comments': 0}
                for article in comments[prj]:
                    for comment in article['comments']:
                        projects[prj]['number_of_comments'] += 1
                        topic = [sum([tw in w for tw in tpc for w in comment.split(' ')]) for tpc in topics]
                        topic_scores.append(topic)
                        comment_sent = sentiment_pred(''.join([ch for ch in comment if ch not in punc]))
                        projects[prj]["comments"].append({'comment': comment, "topic": topic, "sentiment": comment_sent})
                        sentiment += comment_sent
                projects[prj]["topic"] = sum(np.array(topic_scores))
                projects[prj]["sentiment"] = sentiment


In [ ]:
projects = defaultdict(dict)

topic_sentiment_scores(projects)

In [ ]:
pickle.dump(projects, open(dir_ + r"\topics_and_sentiment.pkl", "wb")) 
pickle.dump(topics, open(dir_ + r"\topic_key.pkl", "wb")) 